This file contains some additional baselines used in the paper, namely:
1. PI-EW (plugin)
2. FW val 

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pickle
import os
import csv
import numpy as np
import tensorflow as tf
from sklearn import metrics 

BASE_DIR = '../../../'
import sys
sys.path.append(BASE_DIR)

# custom code
import utils.utils
CONFIG = utils.utils.load_config("../../config.json")

import utils.metrics
import utils.record

Using TensorFlow backend.


In [3]:
DATASET = os.path.basename(os.getcwd()) # name of folder this file is in
RANDOM_SEED = CONFIG['random_seed']
# type of noise
# asym: classes flip to a single other class
# sym: classes flip uniformly to any other class
TYPE = CONFIG["experiment_configs"][DATASET]["type"]
 # chance of flip
NOISE_P = CONFIG["experiment_configs"][DATASET]["noise_p"]
HYPER_VAL_SPLIT = CONFIG["experiment_configs"][DATASET]["hyper_val_split"]

EPOCHS = CONFIG["experiment_configs"][DATASET]["epochs"]
BATCH_SIZE = CONFIG["experiment_configs"][DATASET]["batch_size"]
IMAGE_X = CONFIG["experiment_configs"][DATASET]["image_x_size"]
IMAGE_Y = CONFIG["experiment_configs"][DATASET]["image_y_size"]
IMAGE_SIZE = (IMAGE_Y, IMAGE_X)

print(RANDOM_SEED, TYPE, NOISE_P)

# folders for processed, models
PROCESSED_DIR = os.path.join(BASE_DIR, f'processed/{DATASET}/rs={RANDOM_SEED}')
MODELS_DIR = os.path.join(BASE_DIR, f'models/{DATASET}/rs={RANDOM_SEED}')
RESULTS_DIR = os.path.join(BASE_DIR, 'results')

PROCESSED_SAVEPATH = utils.utils.get_savepath(PROCESSED_DIR, DATASET, ".npz", t=TYPE, np=NOISE_P)
BASE_MODEL_SAVEPATH = utils.utils.get_savepath(MODELS_DIR, DATASET, ".h5", mt="base", t=TYPE, np=NOISE_P)

if not os.path.exists(BASE_MODEL_SAVEPATH):
    print(f"warning: model has not been run for rs={RANDOM_SEED}_t={TYPE}_np={NOISE_P}")

15 asym 0.6


In [4]:
with open(PROCESSED_SAVEPATH, 'rb') as f:
    dat = np.load(f)

    x_hyper_train = dat['x_hyper_train']
    y_hyper_train = dat['y_hyper_train']
    
    x_val = dat['x_val']
    y_val = dat['y_val']
    
    x_hyper_val = dat['x_hyper_val']
    y_hyper_val = dat['y_hyper_val']

    x_test = dat['x_test']
    y_test = dat['y_test']
    

In [5]:
model = utils.utils.make_resnet(
    depth=2,
    random_state=RANDOM_SEED,
    input_shape=(*IMAGE_SIZE, 3),
    nc=10,
)
model.load_weights(BASE_MODEL_SAVEPATH)

In [6]:
preds_val = utils.utils.compute_preds(model, x_val, batch_size=BATCH_SIZE)
preds_hyper_val = utils.utils.compute_preds(model, x_hyper_val, batch_size=BATCH_SIZE)
preds_test = utils.utils.compute_preds(model, x_test, batch_size=BATCH_SIZE)

In [7]:
preds_val_full = np.concatenate([preds_val, preds_hyper_val])
y_val_full = np.concatenate([y_val, y_hyper_val])

In [8]:
METRIC = 'Accuracy'
CLASSES = 10

In [9]:
pred_eta_model = {
    "val_full": preds_val_full,
    "val": preds_val,
    "hyper_val": preds_hyper_val,
    "test": preds_test
}

In [10]:
from utils.papers_piew import plugin

In [11]:
mval_val, mval_test, final_weights = plugin(METRIC, y_val_full, y_test, pred_eta_model, CLASSES)

8640it [00:30, 279.93it/s]                          


In [12]:
os.makedirs(RESULTS_DIR, exist_ok=True)
savepath = os.path.join(RESULTS_DIR, f"results_{DATASET}.csv")
saver = utils.record.Results_Recorder(savepath, DATASET)
saver.save(RANDOM_SEED, METRIC, "plugin", mval_val, mval_val, mval_test, None)
saver.close()

Results file exists, appending to it...
